# Gaussian Process Conditional Density Estimation 
### (Dutordoir, Salimbeni et al. ) Neurips 2018

*(macros here)* 
$%%$
$\newcommand{\N}[]{\mathcal{N}} $
$\newcommand{\log}[]{\text{log }} $
$%%$
$\newcommand{\Y}[]{\mathbf{Y}} $
$\newcommand{\yn}[]{\mathbf{y}_n} $
$\newcommand{\X}[]{\mathbf{X}} $
$\newcommand{\xn}[]{\mathbf{x}_n} $
$\newcommand{\xaug}[]{\mathbf{x}_n'} $
$\newcommand{\W}[]{\mathbf{W}} $
$\newcommand{\wn}[]{\mathbf{w}_n} $
$%%$
$\newcommand{\GP}[]{\mathcal{GP}} $
$\newcommand{\f}[1][\cdot]{\mathbf{f}(#1)} $
$\newcommand{\fl}[1][\cdot]{f_l(#1)} $
$\newcommand{\ul}{\mathbf{u}_l} $
$\newcommand{\m}[1][\cdot]{\mathbf{m}(#1)} $
$\newcommand{\Kxx}{\mathbf{K}^l_{XX}} $
$\newcommand{\k}{k(\cdot, \cdot)} $
$\newcommand{\kz}{\mathbf{k}_Z(\cdot)} $
$\newcommand{\kzT}{\mathbf{k}\T_Z(\cdot)} $
$\newcommand{\Kzz}{\mathbf{K}_{ZZ}} $
$\newcommand{\ml}{\mathbf{m}_{l}} $
$\newcommand{\Sl}{\mathbf{S}_{l}} $
$%%$
$\newcommand{\muwn}{\mathbf{\mu}_{\mathbf{w}_n}} $
$\newcommand{\Lwn}{\mathbf{L}_{\mathbf{w}_n}} $
$%%$
$\newcommand{\P}[]{\mathbf{P}} $
$\newcommand{\A}[]{\mathbf{A}} $
$\newcommand{\I}[]{\mathbf{I}} $
$\newcommand{\d}[]{\text{d}} $
$\newcommand{\g}[]{\mathbf{g}_\phi} $
$\newcommand{\T}[]{^T} $
$%%$
$\newcommand{\E}[]{\mathbb{E}} $
$\newcommand{\Var}[]{\mathbb{V}} $
$\newcommand{\KL}[2]{\text{KL}\big[ #1 \big|\big| #2 \big]} $
$\newcommand{\x}{\mathbf{x}} $


## Model

#### Summary 
1. Data drawn is from multioutput GP, with inputs from an augmented x state: $\xaug = [\A \xn, \wn]$, which is a linear down-projection of $\xn$, concatenated with a auxiliary latent $\wn$
2. The multioutput GP outputs are then up-projected through a mixing matrix $P$, before adding a Gaussian observation noise $\sigma$: $\yn = \N(\P \f[\xaug], \sigma^2 \I)$


#### Detail
$$ p(\Y|\X) = \int p(\Y, \mathbf{f}, \W,  \A, \P |\X)  \d\A \d\P  \d\mathbf{f}\d\W$$
$$ p(\Y, \A, \P, \mathbf{f}, \W |\X)=  p(\f) p(\A) p(\P)  \prod_n p(\yn| \xn,\wn, \f, \A, \P)  p(\wn)$$

where 

$$p(\yn| \xn,\wn,  \f, \A, \P)  = \N( \yn | \P \f[ \lbrack \A\xn,\wn \rbrack], \sigma^2 \I)$$

$$p( \f)  = \prod_l p(\fl) = \prod_l \GP ( \mathbf{0}, \Kxx )$$

$$p( \wn)  = \N (\wn| 0, \I )$$
$$p(\A)  = \prod_{ij} \N(a_{ij} |0, \alpha_A^2) $$
$$p(\P)  = \prod_{ij} \N(p_{ij} |0, \alpha_P^2) $$


## Approximation

We approximate the posterior over latents, $p(\A, \P, \f, \{\wn\} | \Y, \X)$ with a distribution $q(\A, \P, \f, \{\wn\})$.

#### Summary

1. We assume a fully factorised approximation  $q(\A, \P, \f, \{ \wn\} ) =  q(\A) q(\P)q(\f)\prod_n q(\wn)$

2. We chopse $q(\f)$ to be take the form of a multioutput GP posterior, $ q(\f)= \prod_l q(\fl ) =\prod_l \int q(\fl | \ul)q(\ul)\d\ul$. This GP has the same prior as our models, but has additionally been conditioned on some inducing variables $\ul$.

3. We choose the posterior on $q(\wn)$ to be normal $\N(\muwn, (\Lwn\Lwn\T))$ parameterised by a recognition network $\g(\xn, \yn) = [\muwn, \Lwn]$, which is a function of inputs.  (Note we could avoid the gaussian assumption and perform quadrature to find the optimal form of $q(\wn)$ - see paper).

4. We choose the posterior $q(\A)$ to be Gaussian, and could choose the same for $q(\P)$ though in practice, we optimise $\P$ by MAP. 

#### Detail

$$q(\A, \P, \f, \{ \wn\} ) =  q(\A) q(\P)q(\f)\prod_n q(\wn)$$

$$q (\f) = \prod_l q(\fl) =  \prod_l \GP( \kzT\Kzz^{-1}\ml, \k - \kzT\Kzz^{-1}(\Kzz-\Sl)\Kzz^{-1}\kz) $$

$$q (\wn) =  \N(\muwn, (\Lwn\Lwn\T))  \text{ using recognition network } \g(\xn, \yn) = [\muwn, \Lwn]$$

$$q (\A) = \prod_{ij} \N(\mu_{a_{ij}}, \sigma^2_{a_{ij}})$$
$$q (\P) = \prod_{ij} \N(\mu_{p_{ij}}, \sigma^2_{p_{ij}})$$ 

Note in practice we ignore 


#### *GP Detail*

We approximate the posterior on $\fl$ as a posterior GP - a GP from the same prior as $p(\fl)$, but conditioned on pseudo observations $\ul$ at points $\mathbf{Z}$. Note we then marginalise out these observations $q(\ul)$, using a parameterised Gaussian for $q(\ul)$. 

$$q (\f) = \prod_l q(\fl) =\prod_l \int q(\fl | \ul)q(\ul)\d\ul$$

Each $q(\fl | \ul)$ is a (posterior) GP  - one that has been conditioned on pseudo observations $\{\ul\}$ at points $\mathbf{Z}_l$. These are marginalised out, assuming a Gaussian posterior distribution on points for each $q(\ul)$.

$$q(\fl | \ul) = \GP( \kzT\Kzz^{-1}\ul, \k - \kzT\Kzz^{-1}\kz )$$

$$q(\ul) = \N( \ml , \Sl) $$

Using: $\N(x|y)\N(y) = \N\big(\E_{y}[\E_{x|y}], \E_y[\Var_{x|y}] + \Var_y[\E_{x|y}]\big)$

$$q (\f) = \prod_l q(\fl) =  \prod_l \GP( \kzT\Kzz^{-1}\ml, \k - \kzT\Kzz^{-1}(\Kzz-\Sl)\Kzz^{-1}\kz) $$




## Inference

We perform stochastic variational inference by maximising a lower bound to log marginal likelihood, which also corresponds to minimizing a KL divergence between our approximate posterior over latents $Q(\mathcal{Z})$ and true posterior $P(\mathcal{Z}| \X, \Y)$

$$\KL{Q(\mathcal{Z})}{ P(\mathcal{Z}| \X, \Y)}\ge 0$$

$$\log P(\Y | \X) \ge \log P(\Y | \X) -  \int Q(\mathcal{Z})\log  \dfrac{Q(\mathcal{Z})}{ P(\mathcal{Z}| \X, \Y)}\d\mathcal{Z} =: \mathcal{L} $$

$$\mathcal{L}= \int Q(\mathcal{Z})\log\dfrac{P(\mathcal{Z}| \X, \Y)}{  Q(\mathcal{Z})}P(\Y | \X)\d\mathcal{Z} = \int Q(\mathcal{Z}) \log P(\Y | \mathcal{Z}, \X) \d\mathcal{Z} - \KL{Q(\mathcal{Z})}{P(\mathcal{Z})}$$

------

#### First Term: Monte-Carlo Sample

We evaluate our first $\mathcal{L}$ term by sampling, and using the 'reparamaterisation trick'. 


$\int Q(\mathcal{Z}) \log P(\Y | \mathcal{Z}, \X) \d\mathcal{Z} = \int  q(\A) q(\P)q(\f)\prod_n q(\wn)  \log \prod_n p(\yn| \xn,\wn,  \f, \A, \P) \d\A\d\P\d\wn\d\f$

$\int Q(\mathcal{Z}) \log P(\Y | \mathcal{Z}, \X) \d\mathcal{Z} = \sum_n \E \big[ \log \N( \yn | \P \f[ \lbrack \A\xn,\wn \rbrack], \sigma^2 \I)\big]_{q(\A) q(\P)\prod_l q(\fl)\prod_n q(\wn)}$

To calculate an estimate of this term which is differentiable w.r.t to parameters, we can reparameterise a sample from a Gaussian, $\mathbf{x} \sim \N(\mathbf{a}, \mathbf{BB}^T)$, as $\mathbf{x} = \mathbf{a} + \mathbf{B\xi}$ where $\xi \sim \N(\mathbf{0}, \I)$. Since all approximating distributions are Gaussian, this trick is straightforward.

#### Second Term: KL Term

$-\KL{Q(\mathcal{Z})}{P(\mathcal{Z})} = - \KL{q(\A) q(\P)q(\f)\prod_n q(\wn)}{p(\A) p(\P)p(\f)\prod_n p(\wn))}$
$-\KL{Q(\mathcal{Z})}{P(\mathcal{Z})} = - \KL{q(\A)}{p(\A)} - \KL{q(\P)}{p(\P)}- \KL{q(\f)}{p(\f)} - \sum_n\KL{q(\wn)}{p(\wn) }$

NB: KL between two Gaussians result

$\KL{\N(\mu_q, \Sigma_q)}{\N(\mu_p, \Sigma_p)} 
= \int \N(\mu_q, \Sigma_q)\log\dfrac{\N(\mu_q, \Sigma_q)}{\N(\mu_p, \Sigma_p)} \d\mathbf{x} 
= \E\big[\log\N(\mu_q, \Sigma_q) - \log \N(\mu_p, \Sigma_p) \big]_{\N(\mu_q, \Sigma_q)}$

$= \dfrac{1}{2} \big[ (\mu_p - \mu_q)\T \Sigma^{-1}_p (\mu_p - \mu_q) + \text{Tr}\{\Sigma^{-1}_p \Sigma_q\} + \log \dfrac{|\Sigma_p|}{ |\Sigma_q|} - d \big]   $

#### MInibatched Objective

We minibatch our data and  scale the gradients that sum over data points up to the correct size:

$\mathcal{L}_b = \dfrac{N}{N_b}\sum^b_n \E \big[ \log \N( \yn | \P \f[ \lbrack \A\xn,\wn \rbrack], \sigma^2 \I)\big]_{q(\A) q(\P)\prod_l q(\fl)\prod_n q(\wn)} - \KL{q(\A)}{p(\A)} - \KL{q(\P)}{p(\P)}- \KL{q(\f)}{p(\f)} - \sum_n\KL{q(\wn)}{p(\wn) }$

#### KL Derivation:

$ \E\big[\log\N(\mu_q, \Sigma_q) - \log \N(\mu_p, \Sigma_p) \big]_{\N(\mu_q, \Sigma_q)} $

$= \E\big[-\dfrac{1}{2}(\mu_q - \x)\T \Sigma^{-1}_q (\mu_q - \x) + \dfrac{1}{2}(\mu_p - \x)\T \Sigma^{-1}_p (\mu_p - \x) - \dfrac{1}{2}(\log |\Sigma_q| - \log |\Sigma_p| )\big]$

$ = - \dfrac{1}{2}(\log |\Sigma_q| - \log |\Sigma_p| ) + \E\big[-\dfrac{1}{2}\text{Tr}\{\Sigma^{-1}_q (\mu_q - \x)(\mu_q - \x)\T \} + \dfrac{1}{2}(\mu_p - \mu_q + \mu_q - \x)\T \Sigma^{-1}_p (\mu_p  - \mu_q + \mu_q - \x))\big]$

$ = - \dfrac{1}{2}(\log |\Sigma_q| - \log |\Sigma_p| )  -\dfrac{1}{2}\text{Tr}\{\Sigma^{-1}_q\Sigma_q  \} +  \dfrac{1}{2}\E\big[ (\mu_p - \mu_q)\T \Sigma^{-1}_p (\mu_p - \mu_q) + 2 (\mu_p - \mu_q)\T \Sigma^{-1}_p (\mu_q - \x)+ (\mu_q - \x) \T\Sigma^{-1}_p (\mu_q - \x))   \big]$

$ = - \dfrac{1}{2}(\log |\Sigma_q| - \log |\Sigma_p| )  -\dfrac{1}{2}\text{Tr}\{\I\} + \dfrac{1}{2}(\mu_p - \mu_q)\T \Sigma^{-1}_p (\mu_p - \mu_q) + 0 + \text{Tr}\{\Sigma^{-1}_p \Sigma_q\}  $

$= \dfrac{1}{2} \big[ (\mu_p - \mu_q)\T \Sigma^{-1}_p (\mu_p - \mu_q) + \text{Tr}\{\Sigma^{-1}_p \Sigma_q\} + \log \dfrac{|\Sigma_p|}{ |\Sigma_q|} - d \big]   $